In [ ]:
import requests
from PIL import Image
import gradio as gr
import io

# Custom Vision Prediction 키와 URL
PREDICTION_KEY = "1bJyPuHuqMtpDdRrStek0dhNjmoMqQ2aPsMYd72ddnafrmq7rpuBJQQJ99BJACL93NaXJ3w3AAAIACOG9sr0"
ENDPOINT_URL = "https://neptun91cv20251019-prediction.cognitiveservices.azure.com/customvision/v3.0/Prediction/02a54aa3-6104-46cb-8ab5-b00101ba4b48/classify/iterations/Iteration1/image"

headers = {
    "Prediction-Key": PREDICTION_KEY,
    "Content-Type": "application/octet-stream"
}

def predict_with_api(image: Image.Image, top_k: int, show_prob: bool, threshold: float):
    # 이미지를 바이너리로 변환
    buf = io.BytesIO()
    image.save(buf, format='JPEG')
    byte_data = buf.getvalue()
    
    # API 호출
    response = requests.post(ENDPOINT_URL, headers=headers, data=byte_data)
    predictions = response.json()["predictions"]
    
    # 확률 기준 정렬 및 threshold 필터링
    filtered = [p for p in predictions if p["probability"] >= threshold]
    top_predictions = sorted(filtered, key=lambda x: x["probability"], reverse=True)[:top_k]
    
    # 결과 포맷
    text_lines = []
    label_dict = {}
    for pred in top_predictions:
        prob = pred["probability"]
        tag = pred["tagName"]
        if show_prob:
            text_lines.append(f"{tag}: {prob*100:.2f}%")
        else:
            text_lines.append(tag)
        label_dict[tag] = prob
    
    if not text_lines:
        text_lines = ["⚠️ No predictions above threshold."]
    
    return "\n".join(text_lines), label_dict


# Gradio UI
with gr.Blocks() as demo:
    gr.Markdown("## Azure Custom Vision Classifier")
    gr.Markdown("Upload an image to classify using your Azure Custom Vision model.")

    with gr.Row():
        with gr.Column():
            image_input = gr.Image(type="pil", label="Input Image")
            top_k_dropdown = gr.Dropdown([1, 2, 3, 5], value=2, label="Top-K Predictions")
            prob_checkbox = gr.Checkbox(label="Show Prediction Probabilities", value=True)
            threshold_slider = gr.Slider(minimum=0.0, maximum=1.0, value=0.5, step=0.05, label="Confidence Threshold")
            predict_btn = gr.Button("🔍 Predict")

        with gr.Column():
            output_text = gr.Textbox(label="Prediction Text", lines=5)
            output_label = gr.Label(label="Prediction Scores")

    predict_btn.click(
        fn=predict_with_api,
        inputs=[image_input, top_k_dropdown, prob_checkbox, threshold_slider],
        outputs=[output_text, output_label]
    )

demo.launch()

* Running on local URL:  http://127.0.0.1:7868
* To create a public link, set `share=True` in `launch()`.
